# LLM Optimization Modelling Experiment

In [3]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [159]:
problem = '''You are tasked with scheduling the power output of 6 electric power thermal units over the timespan of 15 periods. There is a constant startup cost of for each power unit that is applied if the power plant is turned on. The startup costs are 10324€, 5678€, 7802€, 12899€, 4596€ and 9076€ for powerplants 1 to 6, respectively. In addition, there is a constant shutdown cost for each power unit that is applied if the power plant is turned off. The shutdown costs are 2673€, 5893€, 982€, 6783€, 2596€ and 3561€ for powerplants 1 to 6, respectively. There is also a fixed and variable cost applied if the power plant is running. The fixed cost is constant and the variable cost is proportional to the output of a power plant. There are lower and upper bounds for the output power for each unit. The variable costs are different for each power plant and at each time step. Next, there are maximum power increments and decrements for each power plant that limit how much the output can change from one time period to the next. There is a total power demand that needs to be fulfilled by the power units. Finally, for security reasons, the total available power out should always be 10% higher than the demand. 
'''

## 2. Generate the mathematical model

In [169]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [170]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Power Plant Scheduling

**Parameters:**

* **T:** Number of time periods (T=15).
* **I:** Number of power units (I=6).
* **SU_i:** Startup cost for power unit i (€).
* **SD_i:** Shutdown cost for power unit i (€).
* **FC_i:** Fixed cost for running power unit i (€).
* **VC_i,t:** Variable cost per unit output for power unit i at time t (€/MWh).
* **P_min_i:** Minimum power output for power unit i (MW).
* **P_max_i:** Maximum power output for power unit i (MW).
* **ΔP_up_i:** Maximum power increment for power unit i (MW).
* **ΔP_down_i:** Maximum power decrement for power unit i (MW).
* **D_t:** Total power demand at time t (MW).

**Decision Variables:**

* **P_i,t:** Power output of unit i at time t (MW).
* **U_i,t:** Binary variable, equal to 1 if unit i is running at time t, 0 otherwise.
* **Start_i,t:** Binary variable, equal to 1 if unit i is started up at time t, 0 otherwise.
* **Shut_i,t:** Binary variable, equal to 1 if unit i is shut down at time t, 0 otherwise.

**Objective Function:**

Minimize the total cost over all time periods:

```
Minimize Σ_t=1^T Σ_i=1^I [ FC_i * U_i,t + VC_i,t * P_i,t + SU_i * Start_i,t + SD_i * Shut_i,t ] 
```

**Constraints:**

1. **Power Balance:** Total power output must meet demand plus a 10% security margin.
   ```
   Σ_i=1^I P_i,t >= 1.1 * D_t  for all t = 1,…,T
   ```
2. **Output Limits:** Power output of each unit must be within its limits.
   ```
   P_min_i * U_i,t <= P_i,t <= P_max_i * U_i,t   for all i = 1,…,I and t = 1,…,T
   ```
3. **Ramp Up/Down Limits:** Change in power output between consecutive periods must be within the ramp limits.
   ```
   -ΔP_down_i <= P_i,t - P_i,t-1 <= ΔP_up_i   for all i = 1,…,I and t = 2,…,T
   ```
4. **Startup/Shutdown Logic:**
   ```
   Start_i,t >= U_i,t - U_i,t-1   for all i = 1,…,I and t = 2,…,T
   Shut_i,t >= U_i,t-1 - U_i,t   for all i = 1,…,I and t = 2,…,T
   ```

This model represents a mixed-integer linear programming (MILP) problem, which can be solved using specialized optimization software to find the optimal power output schedule for each unit, minimizing the total operational cost while satisfying the demand and operational constraints. 


## 3. Generate the pyomo code

In [171]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [172]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (replace with actual data)
T = 15 # Number of time periods
I = 6  # Number of power units

SU = {1: 100, 2: 150, 3: 120, 4: 180, 5: 140, 6: 160}  # Startup cost (€)
SD = {1: 50, 2: 75, 3: 60, 4: 90, 5: 70, 6: 80}  # Shutdown cost (€)
FC = {1: 500, 2: 600, 3: 550, 4: 650, 5: 580, 6: 620}  # Fixed cost (€)

# Variable cost (€/MWh) - assuming different cost in each time period
VC = {} 
for i in range(1, I + 1):
    for t in range(1, T + 1):
        VC[i, t] = 10 + i * t * 0.1 

P_min = {1: 10, 2: 15, 3: 12, 4: 18, 5: 14, 6: 16}  # Minimum power output (MW)
P_max = {1: 100, 2: 150, 3: 120, 4: 180, 5: 140, 6: 160}  # Maximum power output (MW)
ΔP_up = {1: 50, 2: 75, 3: 60, 4: 90, 5: 70, 6: 80}  # Maximum power increment (MW)
ΔP_down = {1: 50, 2: 75, 3: 60, 4: 90, 5: 70, 6: 80}  # Maximum power decrement (MW)

# Power demand (MW) - assuming different demand in each time period
D = {}
for t in range(1, T + 1):
    D[t] = 400 + t * 10 

# Create model
model = pyo.ConcreteModel()

# Sets
model.T = pyo.RangeSet(1, T)
model.I = pyo.RangeSet(1, I)

# Variables
model.P = pyo.Var(model.I, model.T, within=pyo.NonNegativeReals)  # Power output
model.U = pyo.Var(model.I, model.T, within=pyo.Binary)  # Unit status
model.Start = pyo.Var(model.I, model.T, within=pyo.Binary)  # Startup
model.Shut = pyo.Var(model.I, model.T, within=pyo.Binary)  # Shutdown

# Objective function
def objective_rule(model):
    return sum(
        FC[i] * model.U[i, t] + VC[i, t] * model.P[i, t] + SU[i] * model.Start[i, t] + SD[i] * model.Shut[i, t]
        for i in model.I
        for t in model.T
    )
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
# Power balance
def power_balance_rule(model, t):
    return sum(model.P[i, t] for i in model.I) >= 1.1 * D[t]
model.power_balance = pyo.Constraint(model.T, rule=power_balance_rule)

# Output limits
def output_limits_rule(model, i, t):
    return P_min[i] * model.U[i, t] <= model.P[i, t] <= P_max[i] * model.U[i, t]
model.output_limits = pyo.Constraint(model.I, model.T, rule=output_limits_rule)

# Ramp up/down limits
def ramp_rule(model, i, t):
    if t > 1:
        return -ΔP_down[i] <= model.P[i, t] - model.P[i, t - 1] <= ΔP_up[i]
    else:
        return pyo.Constraint.Skip
model.ramp = pyo.Constraint(model.I, model.T, rule=ramp_rule)

# Startup/shutdown logic
def startup_rule(model, i, t):
    if t > 1:
        return model.Start[i, t] >= model.U[i, t] - model.U[i, t - 1]
    else:
        return pyo.Constraint.Skip
model.startup = pyo.Constraint(model.I, model.T, rule=startup_rule)

def shutdown_rule(model, i, t):
    if t > 1:
        return model.Shut[i, t] >= model.U[i, t - 1] - model.U[i, t]
    else:
        return pyo.Constraint.Skip
model.shutdown = pyo.Constraint(model.I, model.T, rule=shutdown_rule)

# Solve the model
solver = pyo.SolverFactory('glpk')
solver.solve(model)

# Print results
model.display()
```

This code defines a Pyomo model for the power plant scheduling problem. It includes sample data for the parameters. You will need to replace this sample data with your actual data. The code uses the 'glpk' solver, but you can choose a different solver if you prefer. 

After solving the model, the `model.display()` function will print the optimal values for the decision variables (power output, unit status, startup, and shutdown) for each time period and power unit. 


## 4. Input problem data and try running the generated code

In [173]:
import pyomo.environ as pyo

# Sample Data (replace with actual data)
T = 15 # Number of time periods
I = 6  # Number of power units

SU = {1: 100, 2: 150, 3: 120, 4: 180, 5: 140, 6: 160}  # Startup cost (€)
SD = {1: 50, 2: 75, 3: 60, 4: 90, 5: 70, 6: 80}  # Shutdown cost (€)
FC = {1: 500, 2: 600, 3: 550, 4: 650, 5: 580, 6: 620}  # Fixed cost (€)

# Variable cost (€/MWh) - assuming different cost in each time period
VC = {} 
for i in range(1, I + 1):
    for t in range(1, T + 1):
        VC[i, t] = 10 + i * t * 0.1 

P_min = {1: 10, 2: 15, 3: 12, 4: 18, 5: 14, 6: 16}  # Minimum power output (MW)
P_max = {1: 100, 2: 150, 3: 120, 4: 180, 5: 140, 6: 160}  # Maximum power output (MW)
ΔP_up = {1: 50, 2: 75, 3: 60, 4: 90, 5: 70, 6: 80}  # Maximum power increment (MW)
ΔP_down = {1: 50, 2: 75, 3: 60, 4: 90, 5: 70, 6: 80}  # Maximum power decrement (MW)

# Power demand (MW) - assuming different demand in each time period
D = {}
for t in range(1, T + 1):
    D[t] = 400 + t * 10 

# Create model
model = pyo.ConcreteModel()

# Sets
model.T = pyo.RangeSet(1, T)
model.I = pyo.RangeSet(1, I)

# Variables
model.P = pyo.Var(model.I, model.T, within=pyo.NonNegativeReals)  # Power output
model.U = pyo.Var(model.I, model.T, within=pyo.Binary)  # Unit status
model.Start = pyo.Var(model.I, model.T, within=pyo.Binary)  # Startup
model.Shut = pyo.Var(model.I, model.T, within=pyo.Binary)  # Shutdown

# Objective function
def objective_rule(model):
    return sum(
        FC[i] * model.U[i, t] + VC[i, t] * model.P[i, t] + SU[i] * model.Start[i, t] + SD[i] * model.Shut[i, t]
        for i in model.I
        for t in model.T
    )
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
# Power balance
def power_balance_rule(model, t):
    return sum(model.P[i, t] for i in model.I) >= 1.1 * D[t]
model.power_balance = pyo.Constraint(model.T, rule=power_balance_rule)

# Output limits
def output_limits_rule(model, i, t):
    return P_min[i] * model.U[i, t] <= model.P[i, t] <= P_max[i] * model.U[i, t]
model.output_limits = pyo.Constraint(model.I, model.T, rule=output_limits_rule)

# Ramp up/down limits
def ramp_rule(model, i, t):
    if t > 1:
        return -ΔP_down[i] <= model.P[i, t] - model.P[i, t - 1] <= ΔP_up[i]
    else:
        return pyo.Constraint.Skip
model.ramp = pyo.Constraint(model.I, model.T, rule=ramp_rule)

# Startup/shutdown logic
def startup_rule(model, i, t):
    if t > 1:
        return model.Start[i, t] >= model.U[i, t] - model.U[i, t - 1]
    else:
        return pyo.Constraint.Skip
model.startup = pyo.Constraint(model.I, model.T, rule=startup_rule)

def shutdown_rule(model, i, t):
    if t > 1:
        return model.Shut[i, t] >= model.U[i, t - 1] - model.U[i, t]
    else:
        return pyo.Constraint.Skip
model.shutdown = pyo.Constraint(model.I, model.T, rule=shutdown_rule)

# Solve the model
solver = pyo.SolverFactory('glpk')
solver.solve(model)

# Print results
model.display()

ERROR: Rule failed when generating expression for Constraint output_limits
with index (1, 1): PyomoException: Cannot convert non-constant Pyomo
expression (10*U[1,1]  <=  P[1,1]) to bool. This error is usually caused by
using a Var, unit, or mutable Param in a Boolean context such as an "if"
statement, or when checking container membership or equality. For example,
        >>> m.x = Var()
        >>> if m.x >= 1:
        ...     pass
    and
        >>> m.y = Var()
        >>> if m.y in [m.x, m.y]:
        ...     pass
    would both cause this exception.
ERROR: Constructing component 'output_limits' from data=None failed:
        PyomoException: Cannot convert non-constant Pyomo expression
        (10*U[1,1]  <=  P[1,1]) to bool.
    This error is usually caused by using a Var, unit, or mutable Param in a
    Boolean context such as an "if" statement, or when checking container
    membership or equality. For example,
        >>> m.x = Var()
        >>> if m.x >= 1:
        ...     pa

PyomoException: Cannot convert non-constant Pyomo expression (10*U[1,1]  <=  P[1,1]) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.

## 5. Correct the code to verify model viability (optional)

In [166]:
import pyomo.environ as pyo
### For copying data
# SU = {1: 10324, 2: 5678, 3: 7802, 4: 12899, 5: 4596, 6: 9076}
# SD = {1: 2673, 2: 5893, 3: 982, 4: 6783, 5: 2596, 6: 3561}
# FC = {1: 2000, 2: 3000, 3: 2500, 4: 4000, 5: 3500, 6: 4500} # Sample data
# VC = {(1, 1): 20, (1, 2): 22, (1, 3): 23, (1, 4): 24, (1, 5): 25, (1, 6): 26, (1, 7): 27, (1, 8): 28, (1, 9): 29, (1, 10): 30, (1, 11): 31, (1, 12): 32, (1, 13): 33, (1, 14): 34, (1, 15): 35, (2, 1): 15, (2, 2): 16, (2, 3): 17, (2, 4): 18, (2, 5): 19, (2, 6): 20, (2, 7): 21, (2, 8): 22, (2, 9): 23, (2, 10): 24, (2, 11): 25, (2, 12): 26, (2, 13): 27, (2, 14): 28, (2, 15): 29, (3, 1): 18, (3, 2): 19, (3, 3): 20, (3, 4): 21, (3, 5): 22, (3, 6): 23, (3, 7): 24, (3, 8): 25, (3, 9): 26, (3, 10): 27, (3, 11): 28, (3, 12): 29, (3, 13): 30, (3, 14): 31, (3, 15): 32, (4, 1): 25, (4, 2): 26, (4, 3): 27, (4, 4): 28, (4, 5): 29, (4, 6): 30, (4, 7): 31, (4, 8): 32, (4, 9): 33, (4, 10): 34, (4, 11): 35, (4, 12): 36, (4, 13): 37, (4, 14): 38, (4, 15): 39, (5, 1): 22, (5, 2): 23, (5, 3): 24, (5, 4): 25, (5, 5): 26, (5, 6): 27, (5, 7): 28, (5, 8): 29, (5, 9): 30, (5, 10): 31, (5, 11): 32, (5, 12): 33, (5, 13): 34, (5, 14): 35, (5, 15): 36, (6, 1): 30, (6, 2): 31, (6, 3): 32, (6, 4): 33, (6, 5): 34, (6, 6): 35, (6, 7): 36, (6, 8): 37, (6, 9): 38, (6, 10): 39, (6, 11): 40, (6, 12): 41, (6, 13): 42, (6, 14): 43, (6, 15): 44} # Sample data

# L = {1: 50, 2: 40, 3: 30, 4: 60, 5: 55, 6: 65} # Sample lower bounds
# U = {1: 500, 2:  600, 3: 550, 4: 700, 5: 650, 6: 750}  # Sample upper bounds
# Delta_P_plus = {1: 100, 2: 120, 3: 110, 4: 130, 5: 125, 6: 140}  # Sample max power increment
# Delta_P_minus = {1: 90, 2: 110, 3: 100, 4: 120, 5: 115, 6: 130} # Sample max power decrement
# D = {1: 1000, 2: 1200, 3: 1300, 4: 1100, 5: 1500, 6: 1400, 7: 1600, 8: 1300, 9: 1700, 10: 1800,
#      11: 1900, 12: 1600, 13: 2000, 14: 1800, 15: 1700}  # Sample demand data


###
# Sample data (Replace with actual data)
T = 15  # Number of time periods
N = 6   # Number of power units

SU = {1: 10324, 2: 5678, 3: 7802, 4: 12899, 5: 4596, 6: 9076} # Startup costs
SD = {1: 2673, 2: 5893, 3: 982, 4: 6783, 5: 2596, 6: 3561}  # Shutdown costs
FC = {1: 2000, 2: 3000, 3: 2500, 4: 4000, 5: 3500, 6: 4500}  # Fixed costs

# Variable costs (sample data - replace with actual data)
VC = {(1, 1): 20, (1, 2): 22, (1, 3): 23, (1, 4): 24, (1, 5): 25, (1, 6): 26, (1, 7): 27, (1, 8): 28, (1, 9): 29, (1, 10): 30, (1, 11): 31, (1, 12): 32, (1, 13): 33, (1, 14): 34, (1, 15): 35, (2, 1): 15, (2, 2): 16, (2, 3): 17, (2, 4): 18, (2, 5): 19, (2, 6): 20, (2, 7): 21, (2, 8): 22, (2, 9): 23, (2, 10): 24, (2, 11): 25, (2, 12): 26, (2, 13): 27, (2, 14): 28, (2, 15): 29, (3, 1): 18, (3, 2): 19, (3, 3): 20, (3, 4): 21, (3, 5): 22, (3, 6): 23, (3, 7): 24, (3, 8): 25, (3, 9): 26, (3, 10): 27, (3, 11): 28, (3, 12): 29, (3, 13): 30, (3, 14): 31, (3, 15): 32, (4, 1): 25, (4, 2): 26, (4, 3): 27, (4, 4): 28, (4, 5): 29, (4, 6): 30, (4, 7): 31, (4, 8): 32, (4, 9): 33, (4, 10): 34, (4, 11): 35, (4, 12): 36, (4, 13): 37, (4, 14): 38, (4, 15): 39, (5, 1): 22, (5, 2): 23, (5, 3): 24, (5, 4): 25, (5, 5): 26, (5, 6): 27, (5, 7): 28, (5, 8): 29, (5, 9): 30, (5, 10): 31, (5, 11): 32, (5, 12): 33, (5, 13): 34, (5, 14): 35, (5, 15): 36, (6, 1): 30, (6, 2): 31, (6, 3): 32, (6, 4): 33, (6, 5): 34, (6, 6): 35, (6, 7): 36, (6, 8): 37, (6, 9): 38, (6, 10): 39, (6, 11): 40, (6, 12): 41, (6, 13): 42, (6, 14): 43, (6, 15): 44} # Sample data

P_min = {1: 50, 2: 40, 3: 30, 4: 60, 5: 55, 6: 65}  # Minimum power output
P_max = {1: 500, 2:  600, 3: 550, 4: 700, 5: 650, 6: 750}  # Maximum power output
Delta_P_up = {1: 100, 2: 120, 3: 110, 4: 130, 5: 125, 6: 140}  # Maximum power increment
Delta_P_down =  {1: 90, 2: 110, 3: 100, 4: 120, 5: 115, 6: 130} # Maximum power decrement

# Demand (sample data - replace with actual data)
D = {1: 1000, 2: 1200, 3: 1300, 4: 1100, 5: 1500, 6: 1400, 7: 1600, 8: 1300, 9: 1700, 10: 1800,
     11: 1900, 12: 1600, 13: 2000, 14: 1800, 15: 1700} 

# Create model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.RangeSet(1, N) # Set of power units
model.T = pyo.RangeSet(1, T) # Set of time periods

# Variables
model.P = pyo.Var(model.I, model.T, within=pyo.NonNegativeReals) # Power output
model.U = pyo.Var(model.I, model.T, within=pyo.Binary) # Unit running status
model.S = pyo.Var(model.I, model.T, within=pyo.Binary) # Startup status
model.D = pyo.Var(model.I, model.T, within=pyo.Binary) # Shutdown status

# Objective function
def objective_rule(model):
    return sum(FC[i] * model.U[i, t] + VC[i, t] * model.P[i, t] + SU[i] * model.S[i, t] + SD[i] * model.D[i, t] for i in model.I for t in model.T)
model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
# Power balance
def power_balance_rule(model, t):
    return sum(model.P[i, t] for i in model.I) >= 1.1 * D[t]
model.power_balance = pyo.Constraint(model.T, rule=power_balance_rule)

# Power output limits
def power_output_limits_rule(model, i, t):
    return model.P[i, t] <= P_max[i] * model.U[i, t]
model.power_output_limits = pyo.Constraint(model.I, model.T, rule=power_output_limits_rule)

##Added
def power_output_limits_rule_l(model, i, t):
    return P_min[i] * model.U[i, t] <= model.P[i, t]
model.power_output_limits = pyo.Constraint(model.I, model.T, rule=power_output_limits_rule_l)


# Ramp rate limits
def ramp_up_rule(model, i, t):
    if t > 1:
        return model.P[i, t] - model.P[i, t-1] <= Delta_P_up[i]
    else:
        return pyo.Constraint.Skip
model.ramp_up = pyo.Constraint(model.I, model.T, rule=ramp_up_rule)

def ramp_down_rule(model, i, t):
    if t > 1:
        return -Delta_P_down[i] <= model.P[i, t] - model.P[i, t-1]
    else:
        return pyo.Constraint.Skip
model.ramp_down = pyo.Constraint(model.I, model.T, rule=ramp_down_rule)

# Startup and shutdown logic
def startup_rule(model, i, t):
    if t > 1:
        return model.S[i, t] >= model.U[i, t] - model.U[i, t-1]
    else:
        return pyo.Constraint.Skip
model.startup = pyo.Constraint(model.I, model.T, rule=startup_rule)

def shutdown_rule(model, i, t):
    if t > 1:
        return model.D[i, t] >= model.U[i, t-1] - model.U[i, t]
    else:
        return pyo.Constraint.Skip
model.shutdown = pyo.Constraint(model.I, model.T, rule=shutdown_rule)

# Solve the model
solver = pyo.SolverFactory('glpk')  # Choose a suitable solver
results = solver.solve(model)

# Print results
print(results)
print("Optimal Solution:")
for t in model.T:
    print(f"Time Period {t}:")
    for i in model.I:
        print(f"   Unit {i}: Power Output = {pyo.value(model.P[i, t])}, Running = {pyo.value(model.U[i, t])}, Startup = {pyo.value(model.S[i, t])}, Shutdown = {pyo.value(model.D[i, t])}")

(type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown
with a new Component (type=<class
'pyomo.core.base.constraint.IndexedConstraint'>). This is usually indicative
of a modelling error. To avoid this warning, use block.del_component() and
block.add_component().

Problem: 
- Name: unknown
  Lower bound: 581735.0
  Upper bound: 581735.0
  Number of objectives: 1
  Number of constraints: 441
  Number of variables: 360
  Number of nonzeros: 1110
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.03880453109741211
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Optimal Solution:
Time Period 1:
   Unit 1: Power Output = 0.0, Running = 0.0, Startup = 0.0, Shutdown = 0.0
   Unit 2: Power Output = 1100.0, Running = 0.0, Startup = 0.0, Shutdown = 0.0
   Unit 3: Power Output = 0.0, Runnin

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [174]:
print(response.text)

## Mathematical Optimization Model for Power Plant Scheduling

**Parameters:**

* **T:** Number of time periods (T=15).
* **I:** Number of power units (I=6).
* **SU_i:** Startup cost for power unit i (€).
* **SD_i:** Shutdown cost for power unit i (€).
* **FC_i:** Fixed cost for running power unit i (€).
* **VC_i,t:** Variable cost per unit output for power unit i at time t (€/MWh).
* **P_min_i:** Minimum power output for power unit i (MW).
* **P_max_i:** Maximum power output for power unit i (MW).
* **ΔP_up_i:** Maximum power increment for power unit i (MW).
* **ΔP_down_i:** Maximum power decrement for power unit i (MW).
* **D_t:** Total power demand at time t (MW).

**Decision Variables:**

* **P_i,t:** Power output of unit i at time t (MW).
* **U_i,t:** Binary variable, equal to 1 if unit i is running at time t, 0 otherwise.
* **Start_i,t:** Binary variable, equal to 1 if unit i is started up at time t, 0 otherwise.
* **Shut_i,t:** Binary variable, equal to 1 if unit i is shut down 

In [176]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample Data (replace with actual data)
T = 15 # Number of time periods
I = 6  # Number of power units

SU = {1: 100, 2: 150, 3: 120, 4: 180, 5: 140, 6: 160}  # Startup cost (€)
SD = {1: 50, 2: 75, 3: 60, 4: 90, 5: 70, 6: 80}  # Shutdown cost (€)
FC = {1: 500, 2: 600, 3: 550, 4: 650, 5: 580, 6: 620}  # Fixed cost (€)

# Variable cost (€/MWh) - assuming different cost in each time period
VC = {} 
for i in range(1, I + 1):
    for t in range(1, T + 1):
        VC[i, t] = 10 + i * t * 0.1 

P_min = {1: 10, 2: 15, 3: 12, 4: 18, 5: 14, 6: 16}  # Minimum power output (MW)
P_max = {1: 100, 2: 150, 3: 120, 4: 180, 5: 140, 6: 160}  # Maximum power output (MW)
ΔP_up = {1: 50, 2: 75, 3: 60, 4: 90, 5: 70, 6: 80}  # Maximum power increment (MW)
ΔP_down = {1: 50, 2: 75, 3: 60, 4: 90, 5: 70, 6: 80}  # Maximum power decrement (MW)

# Power demand (MW) - assuming different demand in each time period
D = {}
for t in range(1, T + 1):
    D[t] = 400 + t * 10 

# Cre